Preparación de Datos para Power BI

In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [10]:
tpm = pd.read_csv("../data/processed/transporte_publico_madrid_limpio.csv", parse_dates=['fecha'])
accidentes = pd.read_csv("../data/processed/accidentes_madrid_limpio.csv", parse_dates=['fecha', 'fecha_hora'])


In [29]:
def preparar_datos_power_bi(tpm, accidentes):
    """
    Prepara y exporta los datos en formato óptimo para Power BI
    """
    print("Preparando datos para Power BI...")
    
    # 1. Crear datos de calendario (imprescindible para Power BI)
    rango_fechas = pd.date_range(
        start=min(tpm['fecha'].min(), accidentes['fecha'].min()),
        end=max(tpm['fecha'].max(), accidentes['fecha'].max()),
        freq='D'
    )
    
    calendario = pd.DataFrame({'fecha': rango_fechas})
    calendario['año'] = calendario['fecha'].dt.year
    calendario['mes'] = calendario['fecha'].dt.month
    calendario['trimestre'] = calendario['fecha'].dt.quarter
    calendario['semana'] = calendario['fecha'].dt.isocalendar().week
    calendario['dia_semana'] = calendario['fecha'].dt.day_name()
    calendario['dia_mes'] = calendario['fecha'].dt.day
    calendario['es_fin_semana'] = calendario['dia_semana'].isin(['Saturday', 'Sunday'])
    calendario['mes_nombre'] = calendario['fecha'].dt.month_name()
    
    # 2. Preparar datos de transporte para Power BI
    transporte_pbi = tpm.copy()
    transporte_pbi['año'] = transporte_pbi['fecha'].dt.year
    transporte_pbi['mes'] = transporte_pbi['fecha'].dt.month
    transporte_pbi['mes_nombre'] = transporte_pbi['fecha'].dt.month_name()
    
    # 3. Preparar datos de accidentes para Power BI
    accidentes_pbi = accidentes.copy()
    print(accidentes_pbi.columns)
    accidentes_pbi['año'] = accidentes_pbi['fecha'].dt.year
    accidentes_pbi['mes'] = accidentes_pbi['fecha'].dt.month
    accidentes_pbi['mes_nombre'] = accidentes_pbi['fecha'].dt.month_name()
    
    # 4. Crear tablas de dimensiones (mejor práctica en Power BI)
    dim_transporte = transporte_pbi[['modo_transporte', 'tipo_indicador']].drop_duplicates()
    
    
    dim_distritos = accidentes_pbi[['distrito']].drop_duplicates()

    dim_gravedad = accidentes_pbi[['gravedad', 'lesividad']].drop_duplicates()
    
    # 5. Exportar datos para Power BI
    with pd.ExcelWriter('../data/power_bi/datos_power_bi.xlsx') as writer:
        calendario.to_excel(writer, sheet_name='Calendario', index=False)
        transporte_pbi.to_excel(writer, sheet_name='Transporte', index=False)
        accidentes_pbi.to_excel(writer, sheet_name='Accidentes', index=False)
        dim_transporte.to_excel(writer, sheet_name='Dim_Transporte', index=False)
        dim_distritos.to_excel(writer, sheet_name='Dim_Distritos', index=False)
        dim_gravedad.to_excel(writer, sheet_name='Dim_Gravedad', index=False)
    
    print("Datos exportados exitosamente para Power BI:")
    print("Ubicación: ../data/power_bi/datos_power_bi.xlsx")
    print("Hojas: Calendario, Transporte, Accidentes, Dim_Transporte, Dim_Distritos, Dim_Gravedad")

# Ejecutar preparación de datos
preparar_datos_power_bi(tpm, accidentes)

Preparando datos para Power BI...
Index(['num_expediente', 'fecha', 'hora', 'fecha_hora', 'ano', 'mes',
       'dia_semana', 'hora_dia', 'franja_horaria', 'distrito', 'localizacion',
       'tipo_accidente', 'estado_meteorologico', 'tipo_vehiculo',
       'tipo_persona', 'rango_edad', 'sexo', 'lesividad', 'gravedad',
       'cod_lesividad', 'positiva_alcohol', 'coordenada_x_utm',
       'coordenada_y_utm'],
      dtype='object')
Datos exportados exitosamente para Power BI:
Ubicación: ../data/power_bi/datos_power_bi.xlsx
Hojas: Calendario, Transporte, Accidentes, Dim_Transporte, Dim_Distritos, Dim_Gravedad


In [22]:
def sugerencias_dashboard_principal():
    """
    Sugerencias para el dashboard principal en Power BI
    """
    print("\n" + "="*60)
    print("RECOMENDACIONES PARA DASHBOARD PRINCIPAL")
    print("="*60)
    
    recomendaciones = {
        'KPIs Principales': [
            'Total de accidentes',
            'Tasa de accidentes por 1000 viajeros',
            'Porcentaje de accidentes graves',
            'Evolución mensual comparativa'
        ],
        'Visualizaciones Recomendadas': [
            'Línea de tiempo: Accidentes vs Viajeros (mes a mes)',
            'Mapa de calor por distritos',
            'Gráfico de donut: Distribución por gravedad',
            'Tarjetas con KPIs principales',
            'Filtros por año, mes y distrito'
        ],
        'Métricas Clave': [
            'Crecimiento interanual de accidentes',
            'Distritos con mayor tasa de accidentes',
            'Horarios más peligrosos',
            'Correlación transporte-accidentes'
        ]
    }
    
    for categoria, items in recomendaciones.items():
        print(f"\n🔹 {categoria}:")
        for item in items:
            print(f"   • {item}")

sugerencias_dashboard_principal()


RECOMENDACIONES PARA DASHBOARD PRINCIPAL

🔹 KPIs Principales:
   • Total de accidentes
   • Tasa de accidentes por 1000 viajeros
   • Porcentaje de accidentes graves
   • Evolución mensual comparativa

🔹 Visualizaciones Recomendadas:
   • Línea de tiempo: Accidentes vs Viajeros (mes a mes)
   • Mapa de calor por distritos
   • Gráfico de donut: Distribución por gravedad
   • Tarjetas con KPIs principales
   • Filtros por año, mes y distrito

🔹 Métricas Clave:
   • Crecimiento interanual de accidentes
   • Distritos con mayor tasa de accidentes
   • Horarios más peligrosos
   • Correlación transporte-accidentes


In [23]:
def sugerencias_dashboard_temporal():
    """
    Sugerencias para el dashboard de análisis temporal
    """
    print("\n" + "="*60)
    print("RECOMENDACIONES PARA DASHBOARD TEMPORAL")
    print("="*60)
    
    recomendaciones = {
        'Análisis por Tiempo': [
            'Serie temporal mensual de accidentes',
            'Comparativa año vs año',
            'Estacionalidad por meses',
            'Tendencia con línea de regresión'
        ],
        'Análisis por Horario': [
            'Heatmap: Accidentes por hora y día de semana',
            'Promedio de accidentes por franja horaria',
            'Comparativa fines de semana vs laborables'
        ],
        'Visualizaciones': [
            'Gráfico de líneas con doble eje (accidentes + transporte)',
            'Heatmap calendario (accidentes por día)',
            'Gráfico de área apilada por tipo de accidente',
            'Boxplot: Distribución por horas'
        ]
    }
    
    for categoria, items in recomendaciones.items():
        print(f"\n🔹 {categoria}:")
        for item in items:
            print(f"   • {item}")

sugerencias_dashboard_temporal()


RECOMENDACIONES PARA DASHBOARD TEMPORAL

🔹 Análisis por Tiempo:
   • Serie temporal mensual de accidentes
   • Comparativa año vs año
   • Estacionalidad por meses
   • Tendencia con línea de regresión

🔹 Análisis por Horario:
   • Heatmap: Accidentes por hora y día de semana
   • Promedio de accidentes por franja horaria
   • Comparativa fines de semana vs laborables

🔹 Visualizaciones:
   • Gráfico de líneas con doble eje (accidentes + transporte)
   • Heatmap calendario (accidentes por día)
   • Gráfico de área apilada por tipo de accidente
   • Boxplot: Distribución por horas


In [24]:
def sugerencias_dashboard_geografico():
    """
    Sugerencias para el dashboard geográfico
    """
    print("\n" + "="*60)
    print("RECOMENDACIONES PARA DASHBOARD GEOGRÁFICO")
    print("="*60)
    
    recomendaciones = {
        'Análisis Geográfico': [
            'Mapa de calor por distritos',
            'Top 5 distritos con más accidentes',
            'Top 5 distritos con accidentes más graves',
            'Ratio accidentes/población (si disponible)'
        ],
        'Comparativas por Distrito': [
            'Accidentes por km² (densidad)',
            'Tipo de accidente predominante por zona',
            'Horarios críticos por distrito',
            'Clusterización de distritos similares'
        ],
        'Visualizaciones': [
            'Mapa coroplético de Madrid',
            'Gráfico de barras horizontal por distrito',
            'Scatter plot: Accidentes vs Gravedad por distrito',
            'Tooltip con información detallada por distrito'
        ]
    }
    
    for categoria, items in recomendaciones.items():
        print(f"\n🔹 {categoria}:")
        for item in items:
            print(f"   • {item}")

sugerencias_dashboard_geografico()


RECOMENDACIONES PARA DASHBOARD GEOGRÁFICO

🔹 Análisis Geográfico:
   • Mapa de calor por distritos
   • Top 5 distritos con más accidentes
   • Top 5 distritos con accidentes más graves
   • Ratio accidentes/población (si disponible)

🔹 Comparativas por Distrito:
   • Accidentes por km² (densidad)
   • Tipo de accidente predominante por zona
   • Horarios críticos por distrito
   • Clusterización de distritos similares

🔹 Visualizaciones:
   • Mapa coroplético de Madrid
   • Gráfico de barras horizontal por distrito
   • Scatter plot: Accidentes vs Gravedad por distrito
   • Tooltip con información detallada por distrito


In [26]:
def estructura_dashboard_power_bi():
    """
    Recomendaciones de estructura para el dashboard de Power BI
    """
    print("\n" + "="*60)
    print("ESTRUCTURA RECOMENDADA DEL DASHBOARD")
    print("="*60)
    
    estructura = {
        'Página 1 - Vista General': {
            'Sección Superior': ['KPIs principales', 'Filtros globales'],
            'Sección Central': ['Timeline accidentes vs transporte', 'Mapa de calor distritos'],
            'Sección Inferior': ['Distribución por gravedad', 'Top 5 distritos']
        },
        'Página 2 - Análisis Temporal': {
            'Sección Superior': ['Filtros temporales', 'Evolución mensual'],
            'Sección Central': ['Heatmap por hora/día', 'Comparativa anual'],
            'Sección Inferior': ['Análisis estacional', 'Tendencias']
        },
        'Página 3 - Análisis Geográfico': {
            'Sección Superior': ['Mapa interactivo', 'Filtros geográficos'],
            'Sección Central': ['Clusteres de distritos', 'Análisis por zonas'],
            'Sección Inferior': ['Características por distrito', 'Comparativas']
        },
        'Página 4 - Detalle y Drill-through': {
            'Propósito': 'Página para detalles al hacer clic en otros visuales',
            'Contenido': ['Detalles del accidente', 'Información contextual', 'Gráficos de desglose']
        }
    }
    
    for pagina, secciones in estructura.items():
        print(f"\n📄 {pagina}:")
        for seccion, elementos in secciones.items():
            print(f"   📍 {seccion}:")
            if isinstance(elementos, list):
                for elemento in elementos:
                    print(f"      • {elemento}")
            else:
                print(f"      • {elementos}")

estructura_dashboard_power_bi()


ESTRUCTURA RECOMENDADA DEL DASHBOARD

📄 Página 1 - Vista General:
   📍 Sección Superior:
      • KPIs principales
      • Filtros globales
   📍 Sección Central:
      • Timeline accidentes vs transporte
      • Mapa de calor distritos
   📍 Sección Inferior:
      • Distribución por gravedad
      • Top 5 distritos

📄 Página 2 - Análisis Temporal:
   📍 Sección Superior:
      • Filtros temporales
      • Evolución mensual
   📍 Sección Central:
      • Heatmap por hora/día
      • Comparativa anual
   📍 Sección Inferior:
      • Análisis estacional
      • Tendencias

📄 Página 3 - Análisis Geográfico:
   📍 Sección Superior:
      • Mapa interactivo
      • Filtros geográficos
   📍 Sección Central:
      • Clusteres de distritos
      • Análisis por zonas
   📍 Sección Inferior:
      • Características por distrito
      • Comparativas

📄 Página 4 - Detalle y Drill-through:
   📍 Propósito:
      • Página para detalles al hacer clic en otros visuales
   📍 Contenido:
      • Detalles del ac

In [4]:
def preparar_datos_timeline(tpm, accidentes):
    """
    Prepara datos específicos para el timeline comparativo
    """
    # Datos mensuales de transporte
    transporte_mensual = (tpm[tpm['tipo_indicador'] == 'Viajeros']
                         .groupby(pd.Grouper(key='fecha', freq='M'))['valor']
                         .sum()
                         .reset_index()
                         .rename(columns={'valor': 'viajeros'}))
    
    # Datos mensuales de accidentes
    accidentes_mensual = (accidentes.groupby(pd.Grouper(key='fecha', freq='M'))
                         .size()
                         .reset_index()
                         .rename(columns={0: 'accidentes'}))
    
    # Unir datos
    timeline_data = pd.merge(transporte_mensual, accidentes_mensual, on='fecha', how='inner')
    
    # Exportar para Power BI
    timeline_data.to_csv('../data/power_bi/timeline_comparativo.csv', index=False)
    print("Datos para timeline exportados: ../data/power_bi/timeline_comparativo.csv")

preparar_datos_timeline(tpm, accidentes)

Datos para timeline exportados: ../data/power_bi/timeline_comparativo.csv


C:\Users\ambio\AppData\Local\Temp\ipykernel_15200\648554673.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(pd.Grouper(key='fecha', freq='M'))['valor']
C:\Users\ambio\AppData\Local\Temp\ipykernel_15200\648554673.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  accidentes_mensual = (accidentes.groupby(pd.Grouper(key='fecha', freq='M'))


In [8]:
def preparar_datos_mapa_calor(accidentes):
    """
    Prepara datos para el mapa de calor por distritos
    """
    # Datos de accidentes por distrito
    mapa_data = (accidentes.groupby(['distrito'])
                .agg({
                    'num_expediente': 'count',
                    'cod_lesividad': 'mean',
                    'positiva_alcohol': 'mean'
                })
                .rename(columns={
                    'num_expediente': 'total_accidentes',
                    'cod_lesividad': 'gravedad_promedio',
                    'positiva_alcohol': 'tasa_alcohol'
                })
                .reset_index())
    
    # Coordenadas aproximadas de distritos de Madrid (ejemplo)
    coordenadas_distritos = {
        'Centro': {'lat': 40.415, 'long': -3.703},
        'Salamanca': {'lat': 40.428, 'long': -3.680},
        'Chamartín': {'lat': 40.470, 'long': -3.680},
        'Tetuán': {'lat': 40.459, 'long': -3.697},
        'Chamberí': {'lat': 40.434, 'long': -3.700},
        'Retiro': {'lat': 40.413, 'long': -3.680},
        'Arganzuela': {'lat': 40.395, 'long': -3.697},
        'Usera': {'lat': 40.388, 'long': -3.707},
        'Carabanchel': {'lat': 40.380, 'long': -3.740},
        'Latina': {'lat': 40.388, 'long': -3.743},
        'Villaverde': {'lat': 40.350, 'long': -3.700},
        'Vicálvaro': {'lat': 40.405, 'long': -3.600},
        'San Blas': {'lat': 40.428, 'long': -3.617},
        'Barajas': {'lat': 40.472, 'long': -3.583},
        'Moratalaz': {'lat': 40.407, 'long': -3.640},
        'Ciudad Lineal': {'lat': 40.435, 'long': -3.640},
        'Hortaleza': {'lat': 40.474, 'long': -3.640},
        'Fuencarral-El Pardo': {'lat': 40.498, 'long': -3.717},
        'Moncloa-Aravaca': {'lat': 40.435, 'long': -3.723},
        'Puente de Vallecas': {'lat': 40.392, 'long': -3.623},
        'Villa de Vallecas': {'lat': 40.365, 'long': -3.623}
    }
    
    # Añadir coordenadas
    for distrito, coords in coordenadas_distritos.items():
        mask = mapa_data['distrito'].str.contains(distrito, case=False, na=False)
        if mask.any():
            mapa_data.loc[mask, 'latitud'] = coords['lat']
            mapa_data.loc[mask, 'longitud'] = coords['long']
    
    # Exportar para Power BI
    mapa_data.to_csv('../data/power_bi/mapa_calor_distritos.csv', index=False)
    print("Datos para mapa de calor exportados: ../data/power_bi/mapa_calor_distritos.csv")

preparar_datos_mapa_calor(accidentes)

Datos para mapa de calor exportados: ../data/power_bi/mapa_calor_distritos.csv


In [ ]:
def preparar_datos_temporales(accidentes):
    """
    Prepara datos específicos para el análisis temporal
    """
    print("Preparando datos para análisis temporal...")
    
    # 1. Datos para heatmap por hora/día
    heatmap_data = (accidentes.groupby(['dia_semana', 'hora_dia'])
                   .size()
                   .reset_index(name='total_accidentes')
                   .pivot_table(index='dia_semana', 
                               columns='hora_dia', 
                               values='total_accidentes')
                   .fillna(0))
    
    # Ordenar días de la semana correctamente
    dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    heatmap_data = heatmap_data.reindex(dias_orden)
    
    # 2. Datos para comparativa anual
    comparativa_anual = (accidentes.groupby([accidentes['fecha'].dt.year.rename("ano"), 
                                           accidentes['fecha'].dt.month.rename("mes")])
                        .size()
                        .reset_index(name='total_accidentes'))
    
    # 3. Datos para análisis estacional
    estacional_data = (accidentes.groupby([accidentes['fecha'].dt.month, 
                                         accidentes['gravedad']])
                      .size()
                      .reset_index(name='total_accidentes'))
    estacional_data = estacional_data.rename(columns={'fecha': 'mes'})
    
    # 4. Datos para tendencias
    tendencias_data = (accidentes.resample('W', on='fecha')
                      .size()
                      .reset_index(name='total_accidentes'))
    tendencias_data['media_movil_4semanas'] = tendencias_data['total_accidentes'].rolling(4).mean()
    
    # Exportar todos los datos
    heatmap_data.to_csv('../data/power_bi/heatmap_hora_dia.csv')
    comparativa_anual.to_csv('../data/power_bi/comparativa_anual.csv', index=False)
    estacional_data.to_csv('../data/power_bi/analisis_estacional.csv', index=False)
    tendencias_data.to_csv('../data/power_bi/tendencias_temporales.csv', index=False)
    
    print("Datos para análisis temporal exportados:")
    print("Heatmap hora/día: ../data/power_bi/heatmap_hora_dia.csv")
    print("Comparativa anual: ../data/power_bi/comparativa_anual.csv")
    print("Análisis estacional: ../data/power_bi/analisis_estacional.csv")
    print("Tendencias: ../data/power_bi/tendencias_temporales.csv")
    
    return heatmap_data, comparativa_anual, estacional_data, tendencias_data

# Ejecutar preparación de datos
heatmap, comparativa, estacional, tendencias = preparar_datos_temporales(accidentes)

Preparando datos para análisis temporal...


TypeError: 'method' object is not subscriptable